# Polymorphisms

In [2]:
"""setup"""
from whatever import * 
from whatever import Chain
from toolz.curried import *
from typing import Callable
import toolz
import pytest 
def will_raise(chain, types=TypeError):
    with pytest.raises(types) as t:
        chain.compute()

## Chain

The base class to compose fucntions.

In [3]:
@curry
def compare(composed, composition, value):
    assert composed.compute(value) == composition(value)

In [4]:
"""test_simple"""
f = Chain(10).range.list
compare(f, compose(list, range), 20)
f

In [5]:
f = Chain(10).range(5).list
compare(f, compose(list, partial(range, 5)), 20)
f

In [6]:
f = Chain(10)[range](5)[list]
compare(f, compose(list, partial(range, 5)))
f

## Shorthand

In [7]:
__x(10)

10

In [8]:
g = lambda x: x**2
f = __x(10).range(4).map(g).list
compare(f, compose(list, map(g), partial(range, 4)))(20)

In [9]:
g = lambda x: x**2
f = _x(10).range(4) * g | list
compare(f, compose(list, map(g), partial(range, 4)))(20)
f

[16, 25, 36, 49, 64, 81]

In [10]:
g = lambda x: x**2
f = _x(10).range(4) * g | list
compare(f, compose(list, map(g), partial(range, 4)))(20)
f

[16, 25, 36, 49, 64, 81]

In [11]:
g = lambda x: x**2
f = _x(10).range(3) * g + (lambda x: x< 20) | list
compare(f, compose(list, filter(lambda x: x< 20), map(g), partial(range, 3)))(20)
f

[9, 16]

## Complex Arguments

# Lists

In [12]:
f = _x(10).range[[sum, list, len]]
compare(f, compose(list, juxt(sum, list, len), range))(20)
f

[45, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 10]

In [13]:
f = __x(10).range[(sum, list, len)]
# compare(f, compose(juxt(sum, list, len), range))(20)
f._tokens

[[range, [], ()], [(<function sum>, list, <function len>), [], ()]]

In [14]:
f = _x(10).range | (sum, list, len)
compare(f, compose(juxt(sum, list, len), range))(20)
f

(45, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 10)

In [15]:
f = _x(10).range | [sum, list, len]
compare(f, compose(list, juxt(sum, list, len), range))(20)
f

[45, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 10]

In [16]:
_x(10).range[[sum, list, len]] * type * do(print) | list

<class 'int'>
<class 'list'>
<class 'int'>


[<class 'int'>, <class 'list'>, <class 'int'>]

# Dict

In [17]:
_x(10).range[{'a': sum, 'b': list, 'c': len}]

{'c': 10, 'b': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'a': 45}

In [18]:
_x(10).range | {'a': sum, 'b': list, 'c': len}

{'c': 10, 'b': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'a': 45}

# sets

In [19]:
_x(10).range[{sum, list, len}]

{<built-in function len>: 10, <built-in function sum>: 45, <class 'list'>: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [20]:
d = __x(10).range.list | {sum, list, len} | _this()[sum].compose
d.__()

45

In [21]:
# will_raise(Chain(10).range.map(range).filter(bool).map({'s': sum, 'l': len}).list)

In [25]:
__d = (__x(10).range * range + bool ) * {'s': sum, 'l': len} | list 
assert __d.__() == _x(10).range.map(range).filter(bool).map({'s': sum, 'l': len}).list._()

In [34]:
Chain().range.compute(10)

range(0, 10)

In [35]:
d = (__x(10).range * range + bool ) * [sum, len] | list 
assert d.__() == Chain().range.map(range).filter(bool).map(juxt(sum, len)).map(list).list.compute(10)

In [37]:
f = Chain().range.list.compute(10)
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [38]:
f = _x(10).range.list._()
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [39]:
f = _x(10).range.list._()
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [40]:
__x(10).range.list > identity
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [41]:
f = _x(10) | range | list > identity
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Composition

In [42]:
f = Chain(10).range.list.value
assert isinstance(f, Callable)
f(4)

In [57]:
f = _x(10).range.list._
assert isinstance(f, Callable)
f(4)

[0, 1, 2, 3]

In [58]:
f = _x(10).range.list._
assert isinstance(f, Callable)
f(4)

[0, 1, 2, 3]

In [49]:
f = _x(10).range.list > compose
assert isinstance(f, toolz.functoolz.Compose)
f(4)

[0, 1, 2, 3]

In [50]:
f = _x(10) | range | list > compose
assert isinstance(f, toolz.functoolz.Compose)
f(4)

[0, 1, 2, 3]

# This

In [51]:
class Foo:
    bar = 10
    baz = 'jump'
_this(Foo).bar._()

10

In [52]:
_this({'a': Foo, 'b': 'car'})['a'].bar._()

10

In [54]:
_this({'a': Foo, 'b': 'car'})['b'] | str.upper | interpose('g') | ''.join

'CgAgR'